# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later
import sys
import os
import json
import pandas as pd
sys.path.insert(1, '../../../../src/')
import gt_modules.evaluation as evaluation
from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-sport1-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://gracevirtuoso.dei.unipd.it/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series ("Olympic Games explorative search") 

Consider the following exploratory information need:

> investigate the Olympic Games and find the main BGPs related to these events. Find all the editions of the Summer and Winter Games, compare the sports and disciplines that belong to the games, records and countries.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`       | Human         | node      |
| `wd:Q1189`    | Usain Bolt    | node      |





Also consider

```
wd:Q1189 ?p ?obj .
```

is the BGP to retrieve all **properties of Usain Bolt**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for Olympic Games

2. Return all the editions of the Summer Olympic Games (do not consider future Olympic Games) with the country where they were played (the result set must be a list of elements with edition IRI and label, and country IRI and label).

3. Find the countries which held more than 2 Olympic Games (both Summer and Winter Games, do not consider future Olympic Games) (the result set must be a list of triples country IRI, label and #edition held).

4. Consider the 2008 Summer Olympic Games. Identify all the sports played in this edition (the result set must be a list of couples sport IRI and label).

5. Consider the 2008 Summer Olympic Games. For each sport return the number of different disciplines (the result set must be a list of triples sport IRI, label and #disciplines).

6. Consider the 2008 Summer Olympic Games. Find the top-10 people who won more gold medals (the result set must be a list of triples athlete IRI, label and #gold medal).

7. Find athletes who won at least one gold medal in 4 different editions of the Summer Olympic Games (the result set must be a list of triples athlete IRI, label and #edition).

8. Consider all the edition of the Summer Olympic Games. For each sport return the athlete who won more gold medal (the result set must be a list of elements with sport IRI and label, athlete IRI and label, and #gold medal). 

In [2]:
## startup the evaluation
# setup the file and create the empty json
ipname = "s1.ipynb"
pt = os.getcwd()+os.sep+ipname
evaluation.setup(pt)

The index of this workflow is: 0_5


## Task 1 

In [3]:
# starting from Usain Bolt I look at his properties
queryString = """
SELECT DISTINCT ?pr ?name
WHERE { 
    wd:Q1189 ?pr ?obj.
    ?pr sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1000'), ('name', 'record held')]
[('pr', 'http://www.wikidata.org/prop/direct/P1006'), ('name', 'Nationale Thesaurus voor Auteurs ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1050'), ('name', 'medical condition')]
[('pr', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('pr', 'http://www.wikidata.org/prop/direct/P1146'), ('name', 'World Athletics athlete ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('pr', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), 

In [4]:
# starting from Usain Bolt use the property participant in (P1344) and look for Olympic
queryString = """
SELECT DISTINCT ?obj ?name
WHERE { 
    wd:Q1189 wdt:P1344 ?obj.
    ?obj sc:name ?name .
    FILTER(REGEX(?name,"[O,o]lympic")).
}
"""

print("Results")
x = run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1649415'), ('name', "athletics at the 2012 Summer Olympics – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q2313564'), ('name', "athletics at the 2004 Summer Olympics – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q853003'), ('name', "athletics at the 2008 Summer Olympics – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q1144200'), ('name', "athletics at the 2012 Summer Olympics – men's 4 × 100 metres relay")]
[('obj', 'http://www.wikidata.org/entity/Q25930657'), ('name', "athletics at the 2016 Summer Olympics – men's 4 × 100 metres relay")]
[('obj', 'http://www.wikidata.org/entity/Q26219856'), ('name', "athletics at the 2016 Summer Olympics – men's 200 metres")]
[('obj', 'http://www.wikidata.org/entity/Q25397537'), ('name', "athletics at the 2016 Summer Olympics – men's 100 metres")]
[('obj', 'http://www.wikidata.org/entity/Q734020'), ('name', "athletics at the 2012 Summer Olympics – men's 1

In [5]:
# see the properties of these entities 
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q734020 ?prop ?x.
    ?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('prop', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('prop', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('prop', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('prop', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('prop', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('prop', 'http://www.wikidata.org/pr

In [6]:
# use part of
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q734020 wdt:P361 ?x.
    ?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q185262'), ('name', 'athletics at the 2012 Summer Olympics')]
1


In [7]:
# use part of
queryString = """
SELECT DISTINCT ?y ?name
WHERE { 
    wd:Q734020 wdt:P361 ?x.
    ?x wdt:P361 ?y.
    ?y sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
1


In [8]:
# use the 2012 Summer Olympic
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q8577 wdt:P31 ?x.
    ?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
1


In [9]:
# use the 2012 Summer Olympic
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    wd:Q159821 wdt:P279 ?x.
    ?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q5389'), ('name', 'Olympic Games')]
1


In [10]:
### insert the result of TASK 1 in the file
og_uri = "http://www.wikidata.org/entity/Q5389"
og_name = "Olympic Games"
obj = {"uri":og_uri,"name":og_name}
evaluation.add_result(evaluation.get_index_workflow(pt),"1", evaluation.TYPE_SINGLE ,"uri", [obj] ,"all")

The index of this workflow is: 0_5
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_5.json
JSON object updated


`wd:Q5389` is the Olympic Games

## Task 2

In [11]:
# list the Summer Olympic
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    ?x wdt:P31 wd:Q159821.
    ?x ?prop ?y.
    ?prop sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('prop', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('prop', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('prop', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P1451'), ('name', 'motto text')]
[('prop', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P1617'), ('name', 'BBC Things ID')]
[('prop', 'http

In [12]:
# list the Summer Olympic
queryString = """
SELECT DISTINCT ?x ?name ?loc ?location
WHERE { 
    ?x wdt:P31 wd:Q159821;
        wdt:P17 ?loc;
        wdt:P585 ?y.
    FILTER(year(?y) < 2022).
    ?x sc:name ?name .
    ?loc sc:name ?location.
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q181278'), ('name', '2020 Summer Olympics'), ('loc', 'http://www.wikidata.org/entity/Q17'), ('location', 'Japan')]
[('x', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics'), ('loc', 'http://www.wikidata.org/entity/Q142'), ('location', 'France')]
[('x', 'http://www.wikidata.org/entity/Q8438'), ('name', '1972 Summer Olympics'), ('loc', 'http://www.wikidata.org/entity/Q183'), ('location', 'Germany')]
[('x', 'http://www.wikidata.org/entity/Q8444'), ('name', '1976 Summer Olympics'), ('loc', 'http://www.wikidata.org/entity/Q16'), ('location', 'Canada')]
[('x', 'http://www.wikidata.org/entity/Q8470'), ('name', '1988 Summer Olympics'), ('loc', 'http://www.wikidata.org/entity/Q884'), ('location', 'South Korea')]
[('x', 'http://www.wikidata.org/entity/Q8488'), ('name', '1992 Summer Olympics'), ('loc', 'http://www.wikidata.org/entity/Q29'), ('location', 'Spain')]
[('x', 'http://www.wikidata.org/entity/Q8531'), ('name', '1996 Su

In [13]:
# add the elements to the JSON gt
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    c_uri = i[2][1]
    c_name = i[3][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = ["uri","name"]
    obj["uri"]= c_uri
    obj["name"]= c_name
    objs.append(obj)

evaluation.add_result(evaluation.get_index_workflow(pt),"2", evaluation.TYPE_REFERRED ,["uri","name"], objs)

The index of this workflow is: 0_5
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_5.json
JSON object updated


## Task 3 

In [14]:
# find Summer and Winter Olympic Games
queryString = """
SELECT DISTINCT ?x ?name
WHERE { 
    ?x wdt:P279 wd:Q5389.
    ?x sc:name ?name .
}
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q159821'), ('name', 'Summer Olympic Games')]
[('x', 'http://www.wikidata.org/entity/Q82414'), ('name', 'Winter Olympic Games')]
[('x', 'http://www.wikidata.org/entity/Q188468'), ('name', 'Ancient Olympic Games')]
[('x', 'http://www.wikidata.org/entity/Q988994'), ('name', 'cancelled Olympic Games')]
4


In [15]:
# find Summer and Winter Olympic Games
queryString = """
SELECT DISTINCT ?p ?name
WHERE { 
    VALUES ?games {wd:Q159821 wd:Q82414}.
    ?x wdt:P31 ?games;
        ?p ?d.
    FILTER(isLiteral(?d)).
    FILTER(REGEX(?name,"time") OR REGEX(?name,"date") ).
    ?p sc:name ?name .
}
ORDER BY ?name
"""

print("Results")
x = run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P3999'), ('name', 'date of official closure')]
[('p', 'http://www.wikidata.org/prop/direct/P1619'), ('name', 'date of official opening')]
[('p', 'http://www.wikidata.org/prop/direct/P576'), ('name', 'dissolved, abolished or demolished date')]
[('p', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('p', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('p', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
6


In [16]:
# find Summer and Winter Olympic Games
queryString = """
SELECT DISTINCT ?x ?name YEAR(?d)
WHERE { 
    VALUES ?games {wd:Q159821 wd:Q82414}.
    ?x wdt:P31 ?games;
        wdt:P585 ?d.
    ?x sc:name ?name .
}
ORDER BY ?name
"""

print("Results")
x = run_query(queryString)

Results
[('x', 'http://www.wikidata.org/entity/Q8080'), ('name', '1896 Summer Olympics'), ('callret-2', '1896')]
[('x', 'http://www.wikidata.org/entity/Q8088'), ('name', '1900 Summer Olympics'), ('callret-2', '1900')]
[('x', 'http://www.wikidata.org/entity/Q8098'), ('name', '1904 Summer Olympics'), ('callret-2', '1904')]
[('x', 'http://www.wikidata.org/entity/Q8105'), ('name', '1906 Intercalated Games'), ('callret-2', '1906')]
[('x', 'http://www.wikidata.org/entity/Q8111'), ('name', '1908 Summer Olympics'), ('callret-2', '1908')]
[('x', 'http://www.wikidata.org/entity/Q8118'), ('name', '1912 Summer Olympics'), ('callret-2', '1912')]
[('x', 'http://www.wikidata.org/entity/Q8123'), ('name', '1916 Summer Olympics'), ('callret-2', '1916')]
[('x', 'http://www.wikidata.org/entity/Q8128'), ('name', '1920 Summer Olympics'), ('callret-2', '1920')]
[('x', 'http://www.wikidata.org/entity/Q8132'), ('name', '1924 Summer Olympics'), ('callret-2', '1924')]
[('x', 'http://www.wikidata.org/entity/Q9583

In [17]:
# find Summer and Winter Olympic Games
queryString = """
SELECT DISTINCT ?country ?name COUNT(?x) AS ?olGames
WHERE { 
    VALUES ?games {wd:Q159821 wd:Q82414}.
    ?x wdt:P31 ?games;
        wdt:P585 ?d;
        wdt:P17 ?country.
    ?country sc:name ?name .
    FILTER(?d < NOW()).
}
GROUP BY ?country ?name
HAVING ( COUNT(?x) > 2 )
"""

print("Results")
x = run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q17'), ('name', 'Japan'), ('olGames', '6')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('name', 'United Kingdom'), ('olGames', '3')]
[('country', 'http://www.wikidata.org/entity/Q16'), ('name', 'Canada'), ('olGames', '3')]
[('country', 'http://www.wikidata.org/entity/Q1206012'), ('name', 'German Reich'), ('olGames', '3')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America'), ('olGames', '7')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('name', 'France'), ('olGames', '5')]
6


In [18]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    olGames = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= olGames
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"3", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_5
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_5.json
JSON object updated


## Task 4

In [19]:
# find properties of this edition (2008 edition Q8567)
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q8567 ?prop ?x .
    ?prop sc:name ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('prop', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2397'), ('name', 'YouTube channel ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('prop', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has pa

In [20]:
# find properties of this edition (2008 edition Q8567)
queryString = """
SELECT DISTINCT ?prop ?name
WHERE { 
    wd:Q8567 wdt:P527 ?x .
    ?x ?prop ?y.
    ?prop sc:name ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('prop', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('prop', 'http://www.wikidata.org/prop/direct/P1350'), ('name', 'number of matches played/races/starts')]
[('prop', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('prop', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('prop', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('prop', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('prop', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('prop', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('prop', 'http://www.wikidata.org/prop/direct/P2094'), ('name', 'competition class')]
[('prop', 'http://www.wikidata.org/prop/direct/P2354'), ('name', 'has list')]
[('prop', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'Babe

In [21]:
# use property sport
queryString = """
SELECT DISTINCT ?y ?name
WHERE { 
    wd:Q8567 wdt:P527 ?x .
    ?x wdt:P641 ?y.
    ?y sc:name ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('y', 'http://www.wikidata.org/entity/Q11420'), ('name', 'judo')]
[('y', 'http://www.wikidata.org/entity/Q1455'), ('name', 'field hockey')]
[('y', 'http://www.wikidata.org/entity/Q1734'), ('name', 'volleyball')]
[('y', 'http://www.wikidata.org/entity/Q206989'), ('name', 'shooting sport')]
[('y', 'http://www.wikidata.org/entity/Q2736'), ('name', 'association football')]
[('y', 'http://www.wikidata.org/entity/Q31920'), ('name', 'swimming')]
[('y', 'http://www.wikidata.org/entity/Q32112'), ('name', 'boxing')]
[('y', 'http://www.wikidata.org/entity/Q4543'), ('name', 'beach volleyball')]
[('y', 'http://www.wikidata.org/entity/Q5369'), ('name', 'baseball')]
[('y', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
[('y', 'http://www.wikidata.org/entity/Q542'), ('name', 'athletics')]
[('y', 'http://www.wikidata.org/entity/Q61465'), ('name', 'rhythmic gymnastics')]
[('y', 'http://www.wikidata.org/entity/Q7707'), ('name', 'water polo')]
[('y', 'http://www.wikidata.org/ent

In [22]:
obj = [{"uri":r[0][1],"name":r[1][1]} for r in x]
evaluation.add_result(evaluation.get_index_workflow(pt),"4", evaluation.TYPE_SET ,"uri", obj)

The index of this workflow is: 0_5
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_5.json
JSON object updated


## Task 5

In [23]:
# find properties of this edition (2008 edition Q8567)
queryString = """
SELECT DISTINCT ?sport ?name COUNT(?disc) AS ?disciplines
WHERE { 
    wd:Q8567 wdt:P527 ?x .
    ?x wdt:P527 ?disc;
        wdt:P641 ?sport.
    ?sport sc:name ?name .
}
GROUP BY ?sport ?name
"""

print("Results")
x= run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q31920'), ('name', 'swimming'), ('disciplines', '34')]
[('sport', 'http://www.wikidata.org/entity/Q465014'), ('name', 'trampolining'), ('disciplines', '2')]
[('sport', 'http://www.wikidata.org/entity/Q326827'), ('name', 'artistic gymnastics'), ('disciplines', '14')]
[('sport', 'http://www.wikidata.org/entity/Q3930'), ('name', 'table tennis'), ('disciplines', '4')]
[('sport', 'http://www.wikidata.org/entity/Q7707'), ('name', 'water polo'), ('disciplines', '5')]
[('sport', 'http://www.wikidata.org/entity/Q213934'), ('name', 'canoeing and kayaking'), ('disciplines', '16')]
[('sport', 'http://www.wikidata.org/entity/Q5369'), ('name', 'baseball'), ('disciplines', '1')]
[('sport', 'http://www.wikidata.org/entity/Q902378'), ('name', 'equestrian sport'), ('disciplines', '6')]
[('sport', 'http://www.wikidata.org/entity/Q8418'), ('name', 'handball'), ('disciplines', '4')]
[('sport', 'http://www.wikidata.org/entity/Q11420'), ('name', 'judo'), ('d

In [24]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    disciplines = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= disciplines
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"5", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_5
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_5.json
JSON object updated


## Task 6

In [25]:
# find how people are connected with disciplines
queryString = """
SELECT DISTINCT ?prop ?name 
WHERE { 
    wd:Q8567 wdt:P527 ?x .
    ?x wdt:P527 ?disc;
        wdt:P641 ?sport.
    ?athl wdt:P31 wd:Q5.
    ?athl ?prop ?disc.
    ?prop sc:name ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('prop', 'http://www.wikidata.org/prop/direct/P2522'), ('name', 'victory')]
2


In [26]:
# find how people are connected with disciplines
queryString = """
SELECT DISTINCT ?prop ?name 
WHERE { 
    wd:Q8567 wdt:P527 ?x .
    ?x wdt:P527 ?disc;
        wdt:P641 ?sport.
    ?athl wdt:P31 wd:Q5.
    ?disc ?prop ?athl.
    ?prop sc:name ?name .
}
"""

print("Results")
x= run_query(queryString)

Results
[('prop', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('prop', 'http://www.wikidata.org/prop/direct/P710'), ('name', 'participant')]
[('prop', 'http://www.wikidata.org/prop/direct/P3279'), ('name', 'statistical leader')]
3


In [29]:
# return athletes with their gold medals
queryString = """
SELECT DISTINCT ?athl ?name COUNT(?disc) AS ?gold
WHERE { 
    wd:Q8567 wdt:P527 ?x .
    ?x wdt:P527 ?disc;
        wdt:P641 ?sport.
    ?athl wdt:P31 wd:Q5.
    ?disc wdt:P1346 ?athl.
    ?athl sc:name ?name .
}
GROUP BY ?athl ?name
HAVING (COUNT(?disc) > 1)
ORDER BY DESC (COUNT(?disc))
"""

print("Results")
x= run_query(queryString)

Results
[('athl', 'http://www.wikidata.org/entity/Q39562'), ('name', 'Michael Phelps'), ('gold', '5')]
[('athl', 'http://www.wikidata.org/entity/Q310383'), ('name', 'Chris Hoy'), ('gold', '3')]
[('athl', 'http://www.wikidata.org/entity/Q264399'), ('name', 'Mickaël Bourgain'), ('gold', '2')]
[('athl', 'http://www.wikidata.org/entity/Q270693'), ('name', 'Guo Jingjing'), ('gold', '2')]
[('athl', 'http://www.wikidata.org/entity/Q1189'), ('name', 'Usain Bolt'), ('gold', '2')]
[('athl', 'http://www.wikidata.org/entity/Q193876'), ('name', 'Bradley Wiggins'), ('gold', '2')]
[('athl', 'http://www.wikidata.org/entity/Q227015'), ('name', 'Zou Kai'), ('gold', '2')]
[('athl', 'http://www.wikidata.org/entity/Q228660'), ('name', 'Stephanie Rice'), ('gold', '2')]
[('athl', 'http://www.wikidata.org/entity/Q465367'), ('name', 'Hayden Roulston'), ('gold', '2')]
[('athl', 'http://www.wikidata.org/entity/Q9079'), ('name', 'LaShawn Merritt'), ('gold', '2')]
[('athl', 'http://www.wikidata.org/entity/Q460643'

In [28]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    gold = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= gold
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"6", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_5
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_5.json
JSON object updated


## Task 7

In [30]:
# return athletes who won a gold medals in more than 3 editions
queryString = """
SELECT DISTINCT ?athl ?name COUNT(DISTINCT ?edition) AS ?gold
WHERE { 
    ?edition wdt:P31 wd:Q159821.
    ?edition wdt:P527 ?x .
    ?x wdt:P527 ?disc;
        wdt:P641 ?sport.
    ?athl wdt:P31 wd:Q5.
    ?disc wdt:P1346 ?athl.
    ?athl sc:name ?name .
}
GROUP BY ?athl ?name
HAVING (COUNT(DISTINCT ?edition) > 3)
"""

print("Results")
x= run_query(queryString)

Results
[('athl', 'http://www.wikidata.org/entity/Q233460'), ('name', 'Kaori Icho'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q460643'), ('name', 'Jason Kenny'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q39562'), ('name', 'Michael Phelps'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q131237'), ('name', 'Carl Lewis'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q462790'), ('name', 'Mijaín López'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q449073'), ('name', 'Ben Ainslie'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q270657'), ('name', 'Anna Meares'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q66229'), ('name', 'Jens Fiedler'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q52642'), ('name', 'Al Oerter'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q555418'), ('name', 'Daniel Morelon'), ('gold', '4')]
[('athl', 'http://www.wikidata.org/entity/Q352640'), ('name',

In [31]:
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    gold = i[2][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = "value"
    obj["value"]= gold
    objs.append(obj)
evaluation.add_result(evaluation.get_index_workflow(pt),"7", evaluation.TYPE_REFERRED ,"value", objs)

The index of this workflow is: 0_5
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_5.json
JSON object updated


### Task 8

In [32]:
# find the athletes that have won more gold medals divided by sport
# this is a quite complex solution, but I couldn't manage to find an easier one
# it also returns multiple athletes for each sport since it could happen that they have the same amount of gold medals
queryString = """
SELECT DISTINCT ?sport ?s_name ?athlete ?a_name ?golds
WHERE {
    FILTER(?golds = ?max_golds)
    {
        SELECT DISTINCT ?sport ?s_name ?athlete ?a_name (COUNT(*) as ?golds) ?max_golds
        WHERE {
            ?edition wdt:P31 wd:Q159821 .
            ?event wdt:P361 ?edition .
            ?discipline wdt:P361 ?event ;
                        wdt:P1346 ?athlete ;
                        wdt:P641 ?sport .
            ?athlete sc:name ?a_name .
            {
                SELECT DISTINCT ?sport ?s_name (MAX(?golds) as ?max_golds)
                WHERE {        
                    {
                        SELECT DISTINCT ?sport ?athlete (COUNT(*) AS ?golds)
                        WHERE {
                           ?edition wdt:P31 wd:Q159821 .
                           ?event wdt:P361 ?edition .
                           ?discipline wdt:P361 ?event ;
                                       wdt:P1346 ?athlete ;
                                       wdt:P641 ?sport .

                           # check if Human               
                           ?athlete wdt:P31 wd:Q5 .
                           #labels
                           ?discipline sc:name ?d_name .
                           FILTER(!regex(?d_name, "qualification$"))
                        }
                        GROUP BY ?sport ?athlete
                    }
                    ?sport sc:name ?s_name .
                }
                GROUP BY ?sport ?s_name
            }   
        }
        GROUP BY ?sport ?s_name ?athlete ?a_name ?max_golds
    }
}
ORDER BY DESC(?sport)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sport', 'http://www.wikidata.org/entity/Q902378'), ('s_name', 'equestrian sport'), ('athlete', 'http://www.wikidata.org/entity/Q262338'), ('a_name', 'Nicole Uphoff'), ('golds', '4')]
[('sport', 'http://www.wikidata.org/entity/Q847'), ('s_name', 'tennis'), ('athlete', 'http://www.wikidata.org/entity/Q11459'), ('a_name', 'Serena Williams'), ('golds', '3')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboarding'), ('athlete', 'http://www.wikidata.org/entity/Q107645285'), ('a_name', 'Momiji Nishiya'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboarding'), ('athlete', 'http://www.wikidata.org/entity/Q107351542'), ('a_name', 'Keegan Palmer'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboarding'), ('athlete', 'http://www.wikidata.org/entity/Q66363755'), ('a_name', 'Yūto Horigome'), ('golds', '1')]
[('sport', 'http://www.wikidata.org/entity/Q842284'), ('s_name', 'skateboa

In [33]:
# add the elements to the JSON gt
objs = []
for i in x:
    f_uri = i[0][1]
    f_name = i[1][1]
    c_uri = i[2][1]
    c_name = i[3][1]
    golds = i[4][1]
    obj = {}
    obj["refers_to"] = f_uri
    obj["refers_to_name"] = f_name
    obj["check"] = ["uri","name"]
    obj["uri"]= c_uri
    obj["name"]= c_name
    obj["value"]= golds
    objs.append(obj)

evaluation.add_result(evaluation.get_index_workflow(pt),"8", evaluation.TYPE_REFERRED ,["uri","name"], objs)

The index of this workflow is: 0_5
The path is /locale/data/jupyter/prando/notebook/2022/results/workflow0_5.json
JSON object updated
